In [102]:
import numpy as np
import cv2

import PIL.Image as Image
import os

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras.models import Sequential

import pathlib

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.layers import Concatenate

In [103]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt

In [104]:

IMG_SIZE = (224,224)
IMG_SHAPE = IMG_SIZE + (3,)


data_dir = pathlib.Path('./Image data')
images=list(data_dir.glob('*/*.tif'))

image_dic= {
    'normal' : list(data_dir.glob("e0_Normal/*")),
    'hole':list(data_dir.glob("e1_hole/*")),
    'stain' : list(data_dir.glob("e2_Stain/*")),
    'net' :list(data_dir.glob("e3_Net/*")),
    'color' :list(data_dir.glob("e4_colour/*")),
    'crease': list(data_dir.glob("e5_Crease/*")),
}



labels = {
    'normal':0,
    'hole':1,
    'stain':2,
    'net':3,
    'color':4,
    'crease':5,
}



X,y=[],[]

for name, images in image_dic.items():
    for image in images:
        img = cv2.imread(str(image))
        
        
        if img is None:
            print('Wrong path:', image)
        else:
            resized = cv2.resize(img,IMG_SIZE)
            X.append(resized)
            y.append(labels[name])



X = np.array(X)
y= np.array(y)
X_train, X_test, y_train,y_test = train_test_split(X,y,random_state=42, shuffle=True,test_size=0.3)
X_train = X_train/255
X_test = X_test/255

Wrong path: Image data/e0_Normal/test1.tif.gz
Wrong path: Image data/e0_Normal/test.tif.gz
Wrong path: Image data/e0_Normal/test2.gif.gz


In [105]:
mobile_net= tf.keras.applications.mobilenet.MobileNet()


In [106]:
model_mn = tf.keras.Sequential()
for layes in mobile_net.layers[:-22]:
    model_mn.add(layes)
model_mn.add(tf.keras.layers.GlobalAveragePooling2D())
model_mn.add(tf.keras.layers.Dense(60, activation='tanh'))
model_mn.add(tf.keras.layers.Dropout(0.6))
model_mn.add(tf.keras.layers.Dense(60,activation='tanh',name='predictions'))
model_mn.add(tf.keras.layers.Dense(6, activation='softmax'))



for layers in model_mn.layers[:-19]:
    layers.trainable= False

In [107]:
comb_out = model_mn.layers[-1].output

In [108]:
def submodel(model):
    sep=[]

    for i in range(6):
        res = tf.keras.layers.Dense(1, activation='softmax')(comb_out)
        sep.append(res)

    out = Concatenate()(sep)
    model = tf.keras.Model(mobile_net.input,out)

    return model

In [109]:
model_mn.compile(optimizer = Adam(learning_rate=0.0012), loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['acc'])
model_mn.fit(X_train,to_categorical(y_train),epochs=4,validation_data=(X_test,to_categorical(y_test)))

Epoch 1/4


2023-02-10 12:39:05.120650: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


27/27 [==============================] - ETA: 0s - loss: 1.8569 - acc: 0.2506

2023-02-10 12:39:14.879435: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


27/27 [==============================] - 14s 414ms/step - loss: 1.8569 - acc: 0.2506 - val_loss: 1.6986 - val_acc: 0.2459
Epoch 2/4
27/27 [==============================] - 8s 299ms/step - loss: 1.5603 - acc: 0.3492 - val_loss: 1.5502 - val_acc: 0.3149
Epoch 3/4
27/27 [==============================] - 8s 301ms/step - loss: 1.4137 - acc: 0.4192 - val_loss: 1.5532 - val_acc: 0.3260
Epoch 4/4
27/27 [==============================] - 8s 297ms/step - loss: 1.2882 - acc: 0.4679 - val_loss: 1.4262 - val_acc: 0.3453


In [110]:
model_mn.evaluate(X_test,to_categorical(y_test))

12/12 [==============================] - 2s 184ms/step - loss: 1.4262 - acc: 0.3453


[1.4261924028396606, 0.34530389308929443]